In [63]:
import pandas as pd
import numpy as np

df_game_data_in = pd.read_csv("./raw_data/raw_game_results.csv").drop(['Unnamed: 0'], axis=1)
df_season_results_in = pd.read_csv("./raw_data/raw_season_results.csv").drop(['Unnamed: 0'], axis=1)

df_game_data = df_game_data_in
df_season_results = df_season_results_in

df_game_data has 12 columns. Attendance in NaN for 2020. Appears to be from future games or Covid games in 2020. Games that were walkoffs have W or L followed by wo in the win_loss_result column. Data types are messed up. 

In [64]:
df_game_data.head(-10)
df_game_data[df_game_data["Date"]== '2020-09-06'].head(5)

,Date,Team,H/A,Opponent,win_loss_result,Runs,Runs Allowed,Game Time,Day/Night,Attendance,Game Number,Year
40,2020-09-06,ARI,A,SFG,L,2.0,4.0,2:52,D,NaN,41,2020
99,2020-09-06,ATL,H,WSN,W,10.0,3.0,3:26,D,NaN,40,2020
159,2020-09-06,BAL,H,NYY,W,5.0,1.0,3:02,D,NaN,40,2020
221,2020-09-06,BOS,H,TOR,L,8.0,10.0,4:23,D,NaN,42,2020
280,2020-09-06,CHC,H,STL,L,3.0,7.0,3:17,N,NaN,41,2020


For this analysis I can focus only on games that have a result

In [65]:
df_game_data = df_game_data[df_game_data['win_loss_result'].notna()]
avg_attendance = round(df_game_data.loc[:, 'Attendance'].str.replace(",", "").dropna().astype('int').mean(),0) #will use to impute missing attendance numbers
print(f'average attendance: {avg_attendance}')

average attendance: 26221.0


Need to resolve some data type issues so I can do math. Attendance has a comma that needs to be removed. Win_loss_result has and appended '-wo' for games ending in a walk off, 

In [66]:
#converting attendance into number and setting nulls to the average
# need regex=True for partial string matches
df_game_data = df_game_data.fillna({'Attendance': str(avg_attendance)}).replace({'Attendance': ',', 'win_loss_result': '-wo'}, '', regex=True)

#2020 had legit zero attendance...setting backto zero
df_game_data.loc[df_game_data['Date']<'2021-01-01', 'Attendance'] = 0


Game time also needs to be adjusted. Currently the data is read as H:MM and is an object data type. Going to convert to minutes. Steps to do that include:  
1. split at the delimiter of ':'
2. multiply the first item by 60 
3. add the second item to the first

I'm going to attempt to do this using a custom function

In [67]:
def game_time_to_minutes(game_time):

    if type(game_time) is str: game_time = str(game_time)

    minutes = (float(game_time.split(':')[0]) *60) + float(game_time.split(':')[1])

    return(minutes)

#test = game_time_to_minutes('2:30')
#print(test)


In [68]:
df_game_data['Length Minutes'] = df_game_data['Game Time'].apply(game_time_to_minutes)

In [69]:
data_types = {
    'Date': 'datetime64[ns]',
    'Team': 'string',
    'H/A': 'string',
    'Opponent':'string',
    'win_loss_result':'string',
    'Runs': 'int',
    'Runs Allowed': 'int',
    'Game Time':'string',
    'Day/Night': 'string',
    'Attendance':'float',
    'Game Number': 'int',
    'Year':'int'
}

df_game_data = df_game_data.astype(data_types)
df_game_data.dtypes

Date               datetime64[ns]
Team                       string
H/A                        string
Opponent                   string
win_loss_result            string
Runs                        int32
Runs Allowed                int32
Game Time                  string
Day/Night                  string
Attendance                float64
Game Number                 int32
Year                        int32
Length Minutes            float64
dtype: object

The season results looks good. The GB column -- for teams that won, or are in first in their division. The GB column needs to be changed to 0 for the teams that took 1st

In [70]:
df_season_results

,Season,Team Name,Wins,Losses,Win%,GB,Team
0,2020,Tampa Bay Rays,40,20,0.667,--,TBR
1,2020,New York Yankees,33,27,0.550,7.0,NYY
2,2020,Toronto Blue Jays,32,28,0.533,8.0,TOR
3,2020,Baltimore Orioles,25,35,0.417,15.0,BAL
4,2020,Boston Red Sox,24,36,0.400,16.0,BOS
...,...,...,...,...,...,...,...
175,2025,Los Angeles Dodgers,24,12,0.667,--,LAD
176,2025,San Diego Padres,23,12,0.657,0.5,SDP
177,2025,San Francisco Giants,23,14,0.622,1.5,SFG
178,2025,Arizona Diamondbacks,19,17,0.528,5.0,ARI


In [ ]:
#Handling blanks 
df_season_results.loc[:, 'GB'] = df_season_results.loc[: 'GB'].mask(df_season_results.loc[:, 'GB']=='--', '0')
df_season_results.loc[:, 'Team'] = df_season_results.loc[: 'Team'].mask(df_season_results.loc[:, 'Team Name'].str.contains('Athletics'), 'ATH')
df_season_results.loc[:, 'Team'] = df_season_results.loc[: 'Team'].mask(df_season_results.loc[:, 'Team Name'].str.contains('Cleveland'), 'CLE')

In [75]:
data_types = {
    'Season': int,
    'Team Name': str,
    'Wins': int,
    'Losses': int,
    'Win%': float,
    'GB': float,
    'Team': str
}

df_season_results.astype(data_types)

,Season,Team Name,Wins,Losses,Win%,GB,Team
0,2020,Tampa Bay Rays,40,20,0.667,0.0,TBR
1,2020,New York Yankees,33,27,0.550,7.0,NYY
2,2020,Toronto Blue Jays,32,28,0.533,8.0,TOR
3,2020,Baltimore Orioles,25,35,0.417,15.0,BAL
4,2020,Boston Red Sox,24,36,0.400,16.0,BOS
...,...,...,...,...,...,...,...
175,2025,Los Angeles Dodgers,24,12,0.667,0.0,LAD
176,2025,San Diego Padres,23,12,0.657,0.5,SDP
177,2025,San Francisco Giants,23,14,0.622,1.5,SFG
178,2025,Arizona Diamondbacks,19,17,0.528,5.0,ARI


In [35]:
#join game_results with season results. 
game_data['Previous Year'] = game_data['Year'] - 1

analysis_data = pd.merge(
    game_data[['Game Number', 'Date', 'Team', 'Opponent','Year', 'Previous Year']], 
    season_results[['Season', 'Team', 'Win%']], 
    how='left', 
    left_on=['Opponent', 'Previous Year'], 
    right_on=['Team', 'Season'],   
    ).drop(['Team_y'], axis=1)

analysis_data.head()


,Game Number,Date,Team_x,Opponent,Year,Previous Year,Season,Win%
0,1,2020-07-24,ARI,SDP,2020,2019,NaN,NaN
1,2,2020-07-25,ARI,SDP,2020,2019,NaN,NaN
2,3,2020-07-26,ARI,SDP,2020,2019,NaN,NaN
3,4,2020-07-27,ARI,SDP,2020,2019,NaN,NaN
4,5,2020-07-28,ARI,TEX,2020,2019,NaN,NaN


In [45]:
(analysis_data[(analysis_data['Date'] > '2021-01-01') & (analysis_data['Game Number'] <= 30)]
 .drop(['Previous Year', 'Season'], axis=1)
 .groupby(by = ['Team_x', 'Year'], group_keys=False)
 .agg(
     Mean = ('Win%', np.mean)
 )
 #.agg(Mean = ('Mean', np.max))
 )


Mean
Team_x Year          
ARI    2021  0.512500
       2022  0.493333
       2023  0.541667
       2024  0.478467
       2025  0.514067
...               ...
WSN    2021  0.532833
       2022  0.478000
       2023  0.501833
       2024  0.541296
       2025  0.507167

[150 rows x 1 columns]